# Sampling a posterior

This notebook shows the basic usage of `cogwheel`.

We will choose an event, a waveform approximant and a prior and generate samples from the posterior distribution.

In [ ]:
import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal
lal.swig_redirect_standard_output_error(False)  # Or LAL may run slowly on notebooks

from cogwheel import data
from cogwheel import gw_prior
from cogwheel import likelihood
from cogwheel import sampling
from cogwheel import waveform
from cogwheel.posterior import Posterior

## Event data

By default, `cogwheel.data.EventData` objects are saved in `cogwheel.data.DATADIR` in `npz` format.

In [ ]:
eventnames = sorted(path.name.removesuffix('.npz')
                    for path in data.DATADIR.glob('*.npz'))
print('Available eventnames:')
print(eventnames)

Pre-built `npz` files for some of the events detected up to observing run O3 are available in the [repo](https://github.com/jroulet/cogwheel/tree/main/cogwheel/data); if you are interested in having these you may install `cogwheel` from source.

Otherwise, you can download data and generate `npz` files yourself like so:

In [ ]:
# Create and save `EventData` file
eventname = 'GW150914'
if not data.EventData.get_filename(eventname).exists():
    filenames, detector_names, tgps = data.download_timeseries(eventname)
    event_data = data.EventData.from_timeseries(
        filenames, eventname, detector_names, tgps, t_before=8., fmax=512.)
    event_data.to_npz()

# Update
eventnames = sorted(path.name.removesuffix('.npz')
                    for path in data.DATADIR.glob('*.npz'))

(If `eventname` is unknown to GWOSC, you can also pass a `tgps` argument to `data.download_timeseries`.)

## Approximant options

Pick one from the keys below. Approximants with aligned spins require that the prior has aligned spins as well.

In [ ]:
waveform.APPROXIMANTS

## Prior options

For now, we will choose one of the included `Prior` classes:

In [ ]:
for prior_name, prior_class in gw_prior.prior_registry.items():
    print(f'{prior_name}:\n    {prior_class.__doc__}\n'
          .replace('\n    \n', '\n'))

## Instantiating a `Posterior`

In [ ]:
# Choose from the above options:
eventname = eventnames[0]
mchirp_guess = data.EVENTS_METADATA['mchirp'][eventname]
approximant = 'IMRPhenomXAS'
prior_name = 'AlignedSpinLVCPriorComovingVT'

post = Posterior.from_event(eventname, mchirp_guess, approximant, prior_name)

Things you might want to double-check at this point:

* Did the maximizer find a high likelihood solution ($\log \mathcal{L} \approx \rm{SNR}^2 / 2$)?
* Is the automatically chosen `mchirp_range` ok?
* Are the default `q_min` and `dt0` ok?

In [ ]:
post.likelihood.lnlike(post.likelihood.par_dic_0)

In [ ]:
post.prior.get_init_dict()

In [ ]:
# Say we want to edit the chirp-mass range:
post.prior = post.prior.reinstantiate(mchirp_range=(20, 50))

<div class="alert alert-block alert-info">
<b>Extrinsic parameter marginalization</b>

You can use a likelihood marginalized semi-analytically over distance to remove this dimension from the sampling (recommended).

Instantiate as below. `post_md.likelihood.lnlike` will no longer have an interpretation as $\rm SNR^2 / 2$.

See also the tutorials for full extrinsic-parameter marginalization for signals with [quadrupolar, aligned spin waveforms](https://github.com/jroulet/cogwheel/blob/main/tutorials/factorized_qas.ipynb) or with [precession and higher modes](https://github.com/jroulet/cogwheel/blob/main/tutorials/factorized_phm.ipynb) (even *more* recommended).
</div>

In [ ]:
lookup_table = likelihood.LookupTable()
post_md = Posterior.from_event(eventname, mchirp_guess,
                               approximant='IMRPhenomXPHM',
                               prior_class='MarginalizedDistanceIASPrior',
                               likelihood_kwargs={'lookup_table': lookup_table})

## Instantiating a `Sampler`

The implemented samplers so far are [PyMultiNest](https://johannesbuchner.github.io/PyMultiNest/), [dynesty](https://dynesty.readthedocs.io/en/stable/), [Nautilus](https://nautilus-sampler.readthedocs.io/en/stable/index.html) and [zeus](https://zeus-mcmc.readthedocs.io/en/latest/).

In [ ]:
sampler = sampling.PyMultiNest(post)  # or Dynesty, Nautilus, Zeus 

You can see and edit the options that would be passed to the corresponding sampler by default.

In [ ]:
sampler.run_kwargs_options()

For example, to change the number of live points:

In [ ]:
sampler.run_kwargs['n_live_points'] = 1000

### Running the sampler

In [ ]:
parentdir = 'example'  # Directory that will contain parameter estimation runs

In [ ]:
rundir = sampler.get_rundir(parentdir)

You can run the sampler live or, if you have access to a cluster, submit a job to the workload manager.

In [ ]:
# Run "live" (will take a while):
sampler.run(rundir)

Alternatively, submit job to a scheduler (SLURM, LSF and HTCondor implemented)

    # SLURM
    sampler.submit_slurm(rundir)

    # LSF
    sampler.submit_lsf(rundir)

    # HTCondor
    import os
    sampler.submit_condor(rundir,
                          universe='vanilla',
                          accounting_group=accounting_group, 
                          accounting_group_user=os.environ['USER'])